In [137]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder , MinMaxScaler , StandardScaler , PolynomialFeatures 
from sklearn.linear_model import LinearRegression , Lasso , Ridge , SGDRegressor , ElasticNet
from sklearn.neighbors import KNeighborsRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error , mean_squared_error , r2_score 
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [28]:
train = pd.read_csv("train_v2.csv")


In [29]:
test = pd.read_csv("test_v2.csv")


In [30]:
X_train.isna().sum()

build_year                 0
area                       0
floor                      0
rooms                      0
Parking                    0
Elevator                   0
Warehouse                  0
Luxury_features            0
mean                       0
std                       50
median                     0
amin                       0
amax                       0
neighbour_hood_encoded     0
dtype: int64

In [31]:
X_train.fillna(0 , inplace=True)


# Drop outliers from test

In [32]:
test.describe()

,build_year,area,floor,rooms,Parking,Elevator,Warehouse,Luxury_features,price,mean,std,median,amin,amax,neighbour_hood_encoded
count,5853.000000,5853.000000,5853.000000,5853.000000,5853.000000,5853.000000,5853.000000,5853.000000,5.853000e+03,5.840000e+03,5.821000e+03,5.840000e+03,5.840000e+03,5.840000e+03,5853.000000
mean,1393.184521,139.908252,4.371604,2.389031,0.659320,0.626175,0.588758,0.202973,8.644855e+07,7.800058e+07,2.029221e+07,7.620234e+07,2.482350e+07,1.402287e+08,14.283957
std,7.333277,162.836499,5.887542,0.801909,0.473978,0.483859,0.492101,0.402247,1.263279e+08,2.984214e+07,9.445204e+06,2.881349e+07,1.967649e+07,5.198462e+07,5.322865
min,1362.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+05,1.000000e+07,0.000000e+00,1.000000e+07,8.432200e+05,1.000000e+07,-1.000000
25%,1387.000000,85.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,5.436400e+07,5.934115e+07,1.217463e+07,5.954545e+07,8.783783e+06,9.670000e+07,12.000000
50%,1395.000000,115.000000,3.000000,2.000000,1.000000,1.000000,1.000000,0.000000,7.303371e+07,8.021010e+07,1.699174e+07,7.741900e+07,1.650000e+07,1.313131e+08,16.000000
75%,1400.000000,158.000000,5.000000,3.000000,1.000000,1.000000,1.000000,0.000000,1.000000e+08,9.182152e+07,2.978417e+07,8.928571e+07,4.210000e+07,2.000000e+08,18.000000
max,1402.000000,10000.000000,146.000000,10.000000,1.000000,1.000000,1.000000,1.000000,8.800000e+09,1.516000e+08,5.451752e+07,1.624000e+08,1.350000e+08,2.047000e+08,21.000000


In [33]:
test["area"].sort_values()

1253        1
2801       15
5040       20
1402       20
5780       35
        ...  
1491     1200
2055     1500
275      1533
5622     2900
5582    10000
Name: area, Length: 5853, dtype: int64

In [34]:
np.percentile(test["area"] , 95)

300.0

In [35]:
np.percentile(test["area"] , 0.1)

35.852000000000004

In [36]:
test = test[(test["area"] > 35) &(test["area"] < 300) ].copy()

In [37]:
test["price"].sort_values()

3029        100000
2926       2350000
3285       3000000
1473       3118000
5781       3118000
           ...    
2821     375000000
3722     472727272
821      679577464
5168    1000000000
751     1000000000
Name: price, Length: 5525, dtype: int64

In [38]:
np.percentile(test["price"] , 99)

230200000.0

In [39]:
np.percentile(test["price"] , 1)

12500000.0

In [40]:
test = test[(test["price"] > 12500000.0) &(test["price"] < 230200000.0) ].copy()

In [41]:
test.isna().sum()

build_year                 0
neighbour_hood             3
area                       0
floor                      0
rooms                      0
Parking                    0
Elevator                   0
Warehouse                  0
Luxury_features            0
price                      0
mean                      10
std                       28
median                    10
amin                      10
amax                      10
neighbour_hood_encoded     0
dtype: int64

In [42]:
drop_ind = test[test["neighbour_hood_encoded"]==-1].index
test.drop(drop_ind , axis= 0 , inplace=True)

In [43]:
test.isna().sum()

build_year                 0
neighbour_hood             0
area                       0
floor                      0
rooms                      0
Parking                    0
Elevator                   0
Warehouse                  0
Luxury_features            0
price                      0
mean                       0
std                       18
median                     0
amin                       0
amax                       0
neighbour_hood_encoded     0
dtype: int64

In [44]:
test.fillna(0 , inplace= True)


# Final look at data distributions

In [45]:
test.drop([ "neighbour_hood"] , axis=1).describe()

,build_year,area,floor,rooms,Parking,Elevator,Warehouse,Luxury_features,price,mean,std,median,amin,amax,neighbour_hood_encoded
count,5398.000000,5398.000000,5398.000000,5398.000000,5398.000000,5398.000000,5398.000000,5398.000000,5.398000e+03,5.398000e+03,5.398000e+03,5.398000e+03,5.398000e+03,5.398000e+03,5398.000000
mean,1392.877918,120.848833,4.160430,2.291034,0.645424,0.612449,0.582067,0.183957,7.724850e+07,7.554706e+07,1.942615e+07,7.382069e+07,2.431488e+07,1.367270e+08,14.054279
std,7.401162,51.453840,5.935779,0.706227,0.478429,0.487236,0.493265,0.387485,3.709063e+07,2.815654e+07,9.184905e+06,2.706834e+07,1.925334e+07,5.105301e+07,5.175860
min,1362.000000,36.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.267400e+07,1.000000e+07,0.000000e+00,1.000000e+07,8.432200e+05,1.000000e+07,1.000000
25%,1386.000000,83.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,5.391184e+07,5.809278e+07,1.179333e+07,5.896872e+07,8.700000e+06,9.166667e+07,12.000000
50%,1395.000000,110.000000,3.000000,2.000000,1.000000,1.000000,1.000000,0.000000,7.099400e+07,7.502276e+07,1.643520e+07,7.440000e+07,1.620000e+07,1.250000e+08,16.000000
75%,1400.000000,146.000000,5.000000,3.000000,1.000000,1.000000,1.000000,0.000000,9.187500e+07,8.659810e+07,2.978417e+07,8.460000e+07,4.210000e+07,2.000000e+08,17.000000
max,1402.000000,295.000000,146.000000,5.000000,1.000000,1.000000,1.000000,1.000000,2.300000e+08,1.516000e+08,5.451752e+07,1.624000e+08,1.350000e+08,2.047000e+08,21.000000


In [52]:
drop_ind = test[test["floor"] > 30].index
test.drop(drop_ind , axis= 0 , inplace= True)

In [53]:
train.describe()

,build_year,area,floor,rooms,Parking,Elevator,Warehouse,Luxury_features,price,mean,std,median,amin,amax,neighbour_hood_encoded
count,20881.000000,20881.000000,20881.000000,20881.000000,20881.000000,20881.000000,20881.000000,20881.000000,2.088100e+04,2.088100e+04,2.083100e+04,2.088100e+04,2.088100e+04,2.088100e+04,20881.000000
mean,1392.894354,114.148748,4.232795,2.239213,0.645467,0.613285,0.594129,0.182032,7.303778e+07,7.303778e+07,1.889154e+07,7.134505e+07,2.398720e+07,1.334253e+08,13.670322
std,7.536714,43.082734,5.403487,0.670254,0.478383,0.487009,0.491072,0.385880,3.442528e+07,2.746100e+07,8.855305e+06,2.627741e+07,1.904593e+07,5.075718e+07,5.310036
min,1360.000000,30.000000,-1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,8.432200e+05,2.933375e+06,0.000000e+00,2.895000e+06,8.432200e+05,3.118000e+06,1.000000
25%,1386.000000,81.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,5.180000e+07,5.658122e+07,1.163763e+07,5.787594e+07,8.700000e+06,9.166667e+07,12.000000
50%,1395.000000,110.000000,3.000000,2.000000,1.000000,1.000000,1.000000,0.000000,6.900000e+07,7.381988e+07,1.628758e+07,7.200000e+07,1.595960e+07,1.204000e+08,15.000000
75%,1400.000000,140.000000,5.000000,3.000000,1.000000,1.000000,1.000000,0.000000,8.900000e+07,8.589800e+07,2.646587e+07,8.420000e+07,4.210000e+07,1.949150e+08,17.000000
max,1401.000000,235.000000,146.000000,7.000000,1.000000,1.000000,1.000000,1.000000,2.047000e+08,1.516000e+08,5.451752e+07,1.624000e+08,1.350000e+08,2.047000e+08,21.000000


In [54]:
drop_ind = train[train["floor"] > 30].index
train.drop(drop_ind , axis= 0 , inplace= True)

In [66]:
test.fillna(0 , inplace= True)
train.fillna(0 , inplace= True)

In [67]:
X_test = test.drop(["price" , "neighbour_hood"] , axis=1)
y_test = test["price"]

X_train = train.drop(["price","neighbour_hood"] , axis=1)
y_train = train["price"]

# CV

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import MinMaxScaler , StandardScaler

In [131]:
def eval (model , X , y , cv , verbose = 0):
    result = cross_validate(model ,X= X , y= y  ,return_train_score= True ,cv=cv ,scoring=('r2', 'neg_mean_absolute_error'))
    if verbose and verbose != -1:
        print("r2 for test:" , result["test_r2"])
        print()

        print("r2 for train:" , result["train_r2"])
        print()


        print("mae for test:" , -1*result["test_neg_mean_absolute_error"])
        print()


        print("mae for train:" , -1*result["train_neg_mean_absolute_error"])
        print()
        print("-"*50)

    if verbose != -1:

        print("mean r2 for test:" , np.mean(result["test_r2"]))
        print()
        print("mean r2 for train:" , np.mean(result["train_r2"]))
        print()
        print("mean mae for test:" , np.mean(-1*result["test_neg_mean_absolute_error"]))
        print()
        print("mean mae for train:" , np.mean(-1*result["train_neg_mean_absolute_error"]))
        print()

    return np.mean(-1*result["train_neg_mean_absolute_error"]) , np.mean(-1*result["test_neg_mean_absolute_error"]) , np.mean(result["train_r2"]) ,np.mean(result["test_r2"])  


### Linear Regression

In [134]:
eval(LinearRegression() , X_train , y_train , 5 , verbose= 0 )

mean r2 for test: 0.710185755705514

mean r2 for train: 0.7108375249932

mean mae for test: 13152396.091480574

mean mae for train: 13139354.982888386



(13139354.982888386, 13152396.091480574, 0.7108375249932, 0.710185755705514)

In [118]:
eval(LinearRegression() , MinMaxScaler().fit_transform(X_train) , y_train , 5 , verbose= 0 )

mean r2 for test: 0.7101857557070025

mean r2 for train: 0.7108375249931999

mean mae for test: 13152396.091372095

mean mae for train: 13139354.982784582



In [119]:
eval(LinearRegression() , StandardScaler().fit_transform(X_train) , y_train , 5 , verbose= 0 )

mean r2 for test: 0.7101857557070025

mean r2 for train: 0.7108375249931999

mean mae for test: 13152396.091371965

mean mae for train: 13139354.982784452



### KNN 

### KNN + Poly

In [201]:
from sklearn.preprocessing import PolynomialFeatures
mae_trains_uni = []
mae_tests_uni = []
r2_trains_uni = []
r2_tests_uni = []
for n in [1,3,5,7,9,15,21,31,51,71,91]:
    print(n)
    result =eval(KNeighborsRegressor(n_neighbors = n) , X_train , y_train , 5 , verbose= -1 )
    mae_train , mae_test, r2_train, r2_test = result
    mae_trains_uni.append(mae_train)
    mae_tests_uni.append(mae_test)
    r2_trains_uni.append(r2_train)
    r2_tests_uni.append(r2_test)

1
3


KeyboardInterrupt: 

In [204]:
eval(KNeighborsRegressor(n_neighbors = 3) , PolynomialFeatures(degree=2 , interaction_only=True, include_bias=False).fit_transform(X_train) , y_train , 5 , verbose= 0 )

mean r2 for test: 0.7637272453347976

mean r2 for train: 0.8828074605775165

mean mae for test: 10801054.481663909

mean mae for train: 7321787.433144337



(7321787.433144337, 10801054.481663909, 0.8828074605775165, 0.7637272453347976)

In [207]:
eval(KNeighborsRegressor(n_neighbors = 5) , PolynomialFeatures(degree=2 , interaction_only=True, include_bias=False).fit_transform(X_train) , y_train , 5 , verbose= 0 )

mean r2 for test: 0.7649786649976954

mean r2 for train: 0.847504918792765

mean mae for test: 11071246.783285325

mean mae for train: 8826641.610698989



(8826641.610698989, 11071246.783285325, 0.847504918792765, 0.7649786649976954)

In [209]:
eval(KNeighborsRegressor(n_neighbors = 5 , algorithm="ball_tree" , weights= "distance") , PolynomialFeatures(degree=2 , interaction_only=True, include_bias=False).fit_transform(X_train) , y_train , 5 , verbose= 0 )

mean r2 for test: 0.8096005563040747

mean r2 for train: 0.9968532239098608

mean mae for test: 8825430.507776726

mean mae for train: 307984.575904604



(307984.575904604, 8825430.507776726, 0.9968532239098608, 0.8096005563040747)

In [216]:
from sklearn.feature_selection import SelectKBest 
from sklearn.decomposition import PCA

In [215]:
eval(KNeighborsRegressor(n_neighbors = 5 , algorithm="ball_tree" , weights= "distance") , SelectKBest(k=15).fit_transform(PolynomialFeatures(degree=2 , interaction_only=True, include_bias=False).fit_transform(X_train),y_train) , y_train , 5 , verbose= 0 )

mean r2 for test: 0.7970595712932157

mean r2 for train: 0.9737400394424762

mean mae for test: 8820739.149726823

mean mae for train: 1723023.2826533415



(1723023.2826533415, 8820739.149726823, 0.9737400394424762, 0.7970595712932157)

In [233]:
eval(KNeighborsRegressor(n_neighbors = 5 , algorithm="ball_tree" , weights= "distance") , PCA(12).fit_transform(PolynomialFeatures(degree=2 , interaction_only=True, include_bias=False).fit_transform(X_train),y_train) , y_train , 5 , verbose= 0 )

mean r2 for test: 0.8170196086078452

mean r2 for train: 0.9968583195828872

mean mae for test: 8088182.017096845

mean mae for train: 306918.93438670225



(306918.93438670225, 8088182.017096845, 0.9968583195828872, 0.8170196086078452)

In [249]:
eval(KNeighborsRegressor(n_neighbors = 15 , algorithm="ball_tree" , weights= "distance") , PCA(12).fit_transform(PolynomialFeatures(degree=2 , interaction_only=True, include_bias=False).fit_transform(X_train),y_train) , y_train , 5 , verbose= 0 )

mean r2 for test: 0.8315944584374904

mean r2 for train: 0.9968681164628299

mean mae for test: 7856598.287495447

mean mae for train: 306473.84959672275



(306473.84959672275, 7856598.287495447, 0.9968681164628299, 0.8315944584374904)

# Evaluation

## Train The Model

In [252]:
poly = PolynomialFeatures(degree=2 , include_bias= False , interaction_only= True)
pca = PCA(12)
model = KNeighborsRegressor(n_neighbors= 15 , algorithm="ball_tree" , weights= "distance")

X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

X_train_decomposed = pca.fit_transform(X_train_poly , y=y_train)
X_test_decomposed = pca.transform(X_test_poly )

model.fit(X_train_decomposed ,y_train )


KNeighborsRegressor(algorithm='ball_tree', n_neighbors=15, weights='distance')

## Evaluate The Model

In [254]:
y_pred = model.predict(X_test_decomposed)

0.8318915551425587

In [260]:
acc = 1 - (mean_absolute_error(y_true= y_test , y_pred= y_pred) / np.mean(np.hstack((y_train , y_test))))
acc

0.8901872800400568

In [261]:
print("Accuracy score:" ,acc)
print("r2 score:" ,r2_score(y_true= y_test , y_pred= y_pred))
print("mean_absolute_error:" ,mean_absolute_error(y_true= y_test , y_pred= y_pred))

Accuracy score: 0.8901872800400568
r2 score: 0.8318915551425587
mean_absolute_error: 8112645.7723970385


In [264]:
y_pred_train = model.predict(X_train_decomposed)
acc_train = 1 - (mean_absolute_error(y_true= y_train , y_pred=y_pred_train ) / np.mean(np.hstack((y_train , y_test))))
print("Accuracy score on train:" ,acc_train)
print("r2 score on train:" ,r2_score(y_true= y_train , y_pred= y_pred_train))
print("mean_absolute_error on train:" ,mean_absolute_error(y_true= y_train , y_pred=y_pred_train ))

Accuracy score on train: 0.9952412784672646
r2 score on train: 0.996343872314602
mean_absolute_error on train: 351560.5672880405
